# PYTHON LIBRARIES

LOADING PYTHON LIBRARIES

In [119]:
# dash Libraries
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

# plotly Libraries
import plotly.graph_objects as go
import dash_table

# pandas Libraries
import pandas as pd


# keras Library
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

# numpy Library
import numpy as np

DATA CLEANING AND SCALING

In [120]:
# dataset reading
df_nse = pd.read_csv("data/TATACONSUM.NS_historical_data.csv")
df = pd.read_csv("data/stock_data.csv")

In [121]:
df_nse

,Date,Open,High,Low,Close,Adj Close,Volume
0,1996-01-01,28.000000,28.219999,27.799999,27.805000,16.508287,23500
1,1996-01-02,27.805000,28.000000,27.305000,27.344999,16.235182,23000
2,1996-01-03,27.344999,27.700001,27.500000,27.545000,16.353930,21000
3,1996-01-04,27.545000,27.549999,27.000000,27.495001,16.324242,133000
4,1996-01-05,27.500000,27.500000,27.299999,27.379999,16.255959,114500
...,...,...,...,...,...,...,...
6968,2023-09-21,869.000000,874.000000,861.200012,871.000000,871.000000,1165593
6969,2023-09-22,871.000000,889.849976,870.049988,877.750000,877.750000,1518828
6970,2023-09-25,881.950012,905.599976,870.549988,900.599976,900.599976,2700627
6971,2023-09-26,907.599976,909.900024,891.500000,895.200012,895.200012,2064298


In [122]:
# index formatting
df_nse["Date"]=pd.to_datetime(df_nse.Date,format="%Y-%m-%d")
df_nse.index=df_nse['Date']

In [123]:
# data sorting
data=df_nse.sort_index(ascending=True,axis=0)
new_data=pd.DataFrame(index=range(0,len(df_nse)),columns=['Date','Close'])

In [124]:
for i in range(0,len(data)):
    new_data["Date"][i]=data['Date'][i]
    new_data["Close"][i]=data["Close"][i]

new_data.index=new_data.Date
new_data.drop("Date",axis=1,inplace=True)

dataset=new_data.values

In [125]:
data

,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
1996-01-01,1996-01-01,28.000000,28.219999,27.799999,27.805000,16.508287,23500
1996-01-02,1996-01-02,27.805000,28.000000,27.305000,27.344999,16.235182,23000
1996-01-03,1996-01-03,27.344999,27.700001,27.500000,27.545000,16.353930,21000
1996-01-04,1996-01-04,27.545000,27.549999,27.000000,27.495001,16.324242,133000
1996-01-05,1996-01-05,27.500000,27.500000,27.299999,27.379999,16.255959,114500
...,...,...,...,...,...,...,...
2023-09-21,2023-09-21,869.000000,874.000000,861.200012,871.000000,871.000000,1165593
2023-09-22,2023-09-22,871.000000,889.849976,870.049988,877.750000,877.750000,1518828
2023-09-25,2023-09-25,881.950012,905.599976,870.549988,900.599976,900.599976,2700627


Dataset split to Train and Test

In [126]:
total_sample = dataset.shape[0]
train_sample = int(0.8 * total_sample)

In [127]:
train_sample

5578

In [128]:
# 
train=dataset[0:train_sample,:]
valid=dataset[train_sample:,:]

Scaling of data

In [129]:

scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)

In [130]:
x_train, y_train = [],[]

for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])

In [131]:
x_train,y_train=np.array(x_train),np.array(y_train)
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))

MODEL LOADING

In [132]:
model = load_model("data/saved_lstm_model.h5")

In [133]:
inputs=new_data[len(new_data)-len(valid)-60:].values
inputs=inputs.reshape(-1,1)
inputs=scaler.transform(inputs)

In [134]:
X_test=[]
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test=np.array(X_test)

X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
closing_price=model.predict(X_test)
closing_price=scaler.inverse_transform(closing_price)

44/44 [==============================] - 1s 10ms/step


DATASET SPLIT TO VALIDATION AND TRAINING

In [135]:
train=new_data[:train_sample]
valid=new_data[train_sample:]
# valid['Predictions']=closing_price

In [136]:
valid.loc[:,['Predictions']] = closing_price

In [137]:
valid.head()

,Close,Predictions
Date,,
2018-02-05,273.100006,288.662292
2018-02-06,266.700012,282.564270
2018-02-07,280.149994,276.605591
2018-02-08,280.450012,275.838867
2018-02-09,278.25,276.939636


REGREESION METRICS FOR MODEL

In [138]:
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

In [139]:
mae = mean_absolute_error(valid["Close"], valid["Predictions"])
print("mean_absolute_error",mae)


mse = mean_squared_error(valid["Close"], valid["Predictions"])
print("mean_squared_error",mse)


rmse = np.sqrt(mean_squared_error(valid["Close"], valid["Predictions"]))
print("root_mean_squared_error",rmse)

r2 = r2_score(valid["Close"], valid["Predictions"])
print("r-squared_score",r2)

mean_absolute_error 17.25106714173457
mean_squared_error 517.577379978683
root_mean_squared_error 22.750327030148007
r-squared_score 0.9910330016042257


In [140]:
regression_metrics = {
    'MAE': mae,
    'MSE': mse,
    'RMSE': rmse,
    'R2': r2
}

# DASHBOARD

APP SERVER

In [141]:
app = dash.Dash(__name__)
server = app.server

APP LAYOUT

In [152]:
app.layout = html.Div([
   
    html.H1("Stock Price Analysis Dashboard"),
   
    dcc.Tabs(id="tabs", children=[
		# tab1
		dcc.Tab(label="Model and TATA Stock Analysis",className='custom-tab',children=[
            html.Div([
                html.H2("Actual closing price", className='custom-content'),
				dcc.Graph(
					id="Actual Data",
					figure={
						"data":[
							go.Scatter(
								x=train.index,
								y=train["Close"],
								mode='lines'
							)

						],
						"layout":go.Layout(
							title='Actual Closing Rate(1996-2018)',
							xaxis={'title':'Date'},
							yaxis={'title':'Closing Rate'}
						)
					}
				),
			]),
            
			# 2 panes 
			html.Div([
                # left pane
				html.Div([
            	    html.H2("Model Metrics Table", className='custom-content'),
					dash_table.DataTable(
        				id='metrics-table',
        				columns=[
        			    	{'name': 'Metric', 'id': 'Metric'},
        			    	{'name': 'Value', 'id': 'Value'}
        				],
        				data=[{'Metric': metric, 'Value': value} for metric, value in regression_metrics.items()],
                	),
				], className='pane pane1' ),
            
				# right pane
            	html.Div([
					html.H2("LSTM Predicted closing price", className='custom-content'),
					dcc.Graph(
						id="Predicted-data",
						figure={
							"data":[
								go.Scatter(
									x=valid.index,
									y=valid["Predictions"],
									mode='lines'
								)
							],
							"layout":go.Layout(
								title='Predicted Closing Rate(2018-Present)',
								xaxis={'title':'Date'},
								yaxis={'title':'Closing Rate'}
							)
						}
					),
				], className='pane pane2'),
            ],className='pane'),
			
		]),
        
        # dcc.Tab(label='NSE-TATAGLOBAL Stock Data', className='custom-tab', children=[
		# 	html.Div([
		# 		html.H2("Actual closing price", className='custom-content'),
		# 		dcc.Graph(
		# 			id="Actual Data",
		# 			figure={
		# 				"data":[
		# 					go.Scatter(
		# 						x=train.index,
		# 						y=train["Close"],
		# 						mode='lines'
		# 					)

		# 				],
		# 				"layout":go.Layout(
		# 					title='Actual Closing Rate(2013-2014)',
		# 					xaxis={'title':'Date'},
		# 					yaxis={'title':'Closing Rate'}
		# 				)
		# 			}
		# 		),
		# 		html.H2("LSTM Predicted closing price", className='custom-content'),
		# 		dcc.Graph(
		# 			id="Predicted Data",
		# 			figure={
		# 				"data":[
		# 					go.Scatter(
		# 						x=valid.index,
		# 						y=valid["Predictions"],
		# 						mode='lines'
		# 					)
		# 				],
		# 				"layout":go.Layout(
		# 					title='Predicted Closing Rate(2017-2018)',
		# 					xaxis={'title':'Date'},
		# 					yaxis={'title':'Closing Rate'}
		# 				)
		# 			}
		# 		),
        #         html.H2("Model Metrics Table", className='custom-content'),
        #         dash_table.DataTable(
        # 			id='metrics-table',
        # 			columns=[
        # 			    {'name': 'Metric', 'id': 'Metric'},
        # 			    {'name': 'Value', 'id': 'Value'}
        # 			],
        # 			data=[{'Metric': metric, 'Value': value} for metric, value in regression_metrics.items()]
        #         ),
		# 	])        		
        # ]),

		# tab2
        dcc.Tab(label='NSE-TATAGLOBAL Stock Analysis',className='custom-tab', children=[
            html.Div([
                html.H2("Tata Stock Actual vs Predicted closing price", className='custom-content'),

                dcc.Dropdown(id= 'my-dropdown3',
                             options=[{'label': 'Actual Closing', 'value': 'Close'},
                                      {'label': 'Predicted Closing', 'value': 'Predictions'}],
                            multi=True,value=['Close'], className='dropdown-menu'),
                dcc.Graph(id='closing'),
            ]),
            
			html.Div([
                html.H2("Other Stats", className='custom-content'),
                dcc.Checklist(
        			id='toggle-buttons',
        			options=[
            			{'label': 'Open', 'value': 'Open'},
            			{'label': 'High', 'value': 'High'},
            			{'label': 'Low', 'value': 'Low'},
            			{'label': 'Adj Close', 'value': 'Adj Close'},
        			],
        			value=['Open'],  # Default value is an empty list
                    className='dash-checklist'
    			),
                dcc.Graph(id='toggle-button-output'),			
			]),
            
			html.Div([
                html.H2("TATA GLOBAL stock Volume", className='custom-content'),
                dcc.Graph(
					id="Predicted Data",
					figure={
						"data":[
							go.Scatter(
								x=df_nse.index,
								y=df_nse["Volume"],
								mode='lines'
							)
						],
						"layout":go.Layout(
							title='Trade Quantity for Tata Global Stock over Time',
							xaxis={'title':'Date'},
							yaxis={'title':'Transaction Volume'}
						)
					}
				),
			])
        ]),

		# tab3
        dcc.Tab(label='Stock Analysis (AAPL, TSLA, etc)',className='custom-tab', children=[
            html.Div([
                html.H2("Stocks High vs Lows", className='custom-content'),
              
                dcc.Dropdown(id='my-dropdown',
                             options=[{'label': 'Tesla', 'value': 'TSLA'},
                                      {'label': 'Apple','value': 'AAPL'}, 
                                      {'label': 'Facebook', 'value': 'FB'}, 
                                      {'label': 'Microsoft','value': 'MSFT'}], 
                             multi=True,value=['FB'], className='dropdown-menu'),
                dcc.Graph(id='highlow'),


                html.H2("Stocks Market Volume", className='custom-content'),
                dcc.Dropdown(id='my-dropdown2',
                             options=[{'label': 'Tesla', 'value': 'TSLA'},
                                      {'label': 'Apple','value': 'AAPL'}, 
                                      {'label': 'Facebook', 'value': 'FB'},
                                      {'label': 'Microsoft','value': 'MSFT'}], 
                             multi=True,value=['FB'],className='dropdown-menu'),
                dcc.Graph(id='volume')
            ]),
        ])


    ])
])

APP CALLBACKS

In [143]:
@app.callback(Output('highlow', 'figure'),
              [Input('my-dropdown', 'value')])
def update_graph_highlow(selected_dropdown):
    dropdown = {"TSLA": "Tesla","AAPL": "Apple","FB": "Facebook","MSFT": "Microsoft",}
    trace1 = []
    trace2 = []
    for stock in selected_dropdown:
        trace1.append(
          go.Scatter(x=df[df["Stock"] == stock]["Date"],
                     y=df[df["Stock"] == stock]["High"],
                     mode='lines', opacity=0.7, 
                     name=f'High {dropdown[stock]}',textposition='bottom center'))
        trace2.append(
          go.Scatter(x=df[df["Stock"] == stock]["Date"],
                     y=df[df["Stock"] == stock]["Low"],
                     mode='lines', opacity=0.6,
                     name=f'Low {dropdown[stock]}',textposition='bottom center'))
    traces = [trace1, trace2]
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data,
              'layout': go.Layout(colorway=["#5E0DAC", '#FF4F00', '#375CB1', 
                                            '#FF7400', '#FFF400', '#FF0056'],
            height=600,
            title=f"High and Low Prices for {', '.join(str(dropdown[i]) for i in selected_dropdown)} Over Time",
            xaxis={"title":"Date",
                   'rangeselector': {'buttons': list([{'count': 1, 'label': '1M', 
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'count': 6, 'label': '6M', 
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'step': 'all'}])},
                   'rangeslider': {'visible': True}, 'type': 'date'},
             yaxis={"title":"Price (USD)"})}
    return figure

In [144]:
@app.callback(Output('closing','figure'),
              [Input('my-dropdown3', 'value')])
def update_graph_closing(selected_dropdown_close):
    dropdown= {"Close": "Actual","Predictions": "Predicted"}
    trace1=[]
    for stock in selected_dropdown_close:
        trace1.append(
            go.Scatter(x=valid.index,
                       y=valid[stock],
                       mode = 'lines', opacity=0.7,
                       name=f'{dropdown[stock]} Closing ', textposition='bottom center'))
        
    traces = [trace1]
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data, 
              'layout': go.Layout(colorway=["#5E0DAC", '#FF4F00', '#375CB1', 
                                            '#FF7400', '#FFF400', '#FF0056'],
            height=600,
            title=f"{''.join(str(dropdown[i]) for i in selected_dropdown_close)} Prices Over Time",
            xaxis={"title":"Date",
                   'rangeselector': {'buttons': list([{'count': 1, 'label': '1M', 
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'count': 6, 'label': '6M',
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'step': 'all'}])},
                   'rangeslider': {'visible': True}, 'type': 'date'},
             yaxis={"title":"Cosing Price"})}
    return figure

In [145]:
@app.callback(Output('volume', 'figure'),
              [Input('my-dropdown2', 'value')])
def update_graph_volume(selected_dropdown_value):
    dropdown = {"TSLA": "Tesla","AAPL": "Apple","FB": "Facebook","MSFT": "Microsoft",}
    trace1 = []
    for stock in selected_dropdown_value:
        trace1.append(
          go.Scatter(x=df[df["Stock"] == stock]["Date"],
                     y=df[df["Stock"] == stock]["Volume"],
                     mode='lines', opacity=0.7,
                     name=f'Volume {dropdown[stock]}', textposition='bottom center'))
    traces = [trace1]
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data, 
              'layout': go.Layout(colorway=["#5E0DAC", '#FF4F00', '#375CB1', 
                                            '#FF7400', '#FFF400', '#FF0056'],
            height=600,
            title=f"Market Volume for {', '.join(str(dropdown[i]) for i in selected_dropdown_value)} Over Time",
            xaxis={"title":"Date",
                   'rangeselector': {'buttons': list([{'count': 1, 'label': '1M', 
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'count': 6, 'label': '6M',
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'step': 'all'}])},
                   'rangeslider': {'visible': True}, 'type': 'date'},
             yaxis={"title":"Transactions Volume"})}
    return figure

In [146]:
@app.callback(Output('toggle-button-output', 'figure'),
              [Input('toggle-buttons', 'value')])
def update_output(selected_values):
    dropdown = {"Open": "Open","High": "High","Low": "Low","Adj Close": "Adj Close",}
    trace1 = []
    for stock in selected_values:
        trace1.append(
          go.Scatter(x=df_nse.index,
                       y=df_nse[stock],
                       mode = 'lines', opacity=0.7,
                       name=f'{dropdown[stock]} ', textposition='bottom center'))
    traces = [trace1]
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data, 
              'layout': go.Layout(colorway=["#5E0DAC", '#FF4F00', '#375CB1', 
                                            '#FF7400', '#FFF400', '#FF0056'],
            height=600,
            title=f"{', '.join(str(dropdown[i]) for i in selected_values)} values Over Time",
            xaxis={"title":"Date",
                   'rangeselector': {'buttons': list([{'count': 1, 'label': '1M', 
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'count': 6, 'label': '6M',
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'step': 'all'}])},
                   'rangeslider': {'visible': True}, 'type': 'date'},
             yaxis={"title":"Prices"})}
    return figure

APP SERVER START

In [147]:
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)